In [ ]:
from lmfit import Parameters, minimize, report_fit
import math

#estimate the mass of the tuning fork (Needed for fitting the data)
L = 3.570e-3  # m
T = 567.9e-6  # m
W = 314.4e-6  # m
rhoq = 2659  # kgm^-3
# Properties
V = 0.24267 * T * W * L
S = 2 * L * (W + T)
M = V * rhoq

#General shape of a resonance
def genlorxoff(x,A,f_0,width,angle,offset):
    ypart=(A*(((x**2)*width)/((((f_0**2))-((x)**2))**2+(x*width)**2)))
    xpart=(A*((x*(((f_0**2))-((x)**2)))/((((f_0**2))-((x)**2))**2+(x*width)**2)))
    newx=xpart*np.cos(angle)+ypart*-np.sin(angle)-offset
    return(newx)
def genloryoff(x,A,f_0,width,angle,offset):
    ypart=(A*(((x**2)*width)/((((f_0**2))-((x)**2))**2+(x*width)**2)))
    xpart=(A*((x*(((f_0**2))-((x)**2)))/((((f_0**2))-((x)**2))**2+(x*width)**2)))
    newy=xpart*np.sin(angle+np.pi)+ypart*np.cos(angle+np.pi)-offset
    return(newy)

# Define the model functions for fitting both x and y at the same time
def Lorxtofit(x, params):
    m=M
    f_0 = params['f_0'].value
    A = params['A'].value
    A2=A/m
    width = params['width'].value
    angle = params['angle'].value
    offsetx = params['offsetx'].value
    ypart = A * (((x**2) * width) / ((((f_0**2)) - ((x)**2))**2 + (x * width)**2))
    xpart = A * ((x * (((f_0**2)) - ((x)**2))) / ((((f_0**2)) - ((x)**2))**2 + (x * width)**2))
    newx = xpart * np.cos(angle) + ypart * -np.sin(angle) - offsetx
    return newx

def Lorytofit(x, params):
    m=M
    f_0 = params['f_0'].value
    A = params['A'].value
    A2=A/m
    width = params['width'].value
    angle = params['angle'].value
    offsety = params['offsety'].value
    ypart = A * (((x**2) * width) / ((((f_0**2)) - ((x)**2))**2 + (x * width)**2))
    xpart = A * ((x * (((f_0**2)) - ((x)**2))) / ((((f_0**2)) - ((x)**2))**2 + (x * width)**2))
    newy=xpart*np.sin(angle+np.pi)+ypart*np.cos(angle+np.pi)-offsety
    return newy

# Define the objective function to minimize eg the minimisation of the differance in the models to the x and y data combined
def objective(params, x1, y1, x2, y2, y1_err, y2_err):
    res1 = np.abs((y1 - Lorxtofit(x1, params)) / y1_err)
    res2 = np.abs((y2 - Lorytofit(x2, params)) / y2_err)
    return np.concatenate((res1, res2))

In [1]:
#define a version of fitting with and without plotting
def fittingforfreqfig(x,y,f):
    f_0 = 32.755e3
    A = 6.040484330735126e-5
    width = 4
    offset = 0
    angle = np.pi
    ############## param guess ###############
    div=10000 # I haven't added a way to record errors yet but this fitting method needs some values
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
    ax1.errorbar(f, x,xerr=f/div,yerr=np.abs(x)/div,label="xdata",fmt='.b')
    ax1.errorbar(f, y,xerr=f/div,yerr=np.abs(y)/div,label="ydata",fmt='.y')
    ax1.plot(f,genlorxoff(f,A,f_0,width,angle,offset),label="fit",c="c")
    ax1.plot(f,genloryoff(f,A,f_0,width,angle,offset),label="fit",c="r")
    ax1.legend()
    
    # Initialize the parameters
    params = Parameters()
    params.add('f_0', value=f_0)
    params.add('A', value=A)
    params.add('width', value=width, min=0)
    params.add('angle', value=angle,min=0,max=2*np.pi)
    params.add('offsetx', value=offset)
    params.add('offsety', value=offset)

    x1=np.array(list(filter(lambda x: not math.isnan(x), f)))
    y1=np.array(list(filter(lambda x: not math.isnan(x), x)))
    x2=x1
    y2=np.array(list(filter(lambda x: not math.isnan(x), y)))
    y1_err=np.array(list(filter(lambda x: not math.isnan(x), x/div)))
    y2_err=np.array(list(filter(lambda x: not math.isnan(x), x/div)))

    # Perform the simultaneous fit
    result = minimize(objective, params, args=(x1, y1, x2, y2, y1_err, y2_err))

    fit_params = result.params
    
    ax2.errorbar(f, x,xerr=f/div,yerr=np.abs(x)/div,label="xdata",fmt='.b')
    ax2.errorbar(f, y,xerr=f/div,yerr=np.abs(y)/div,label="ydata",fmt='.y')
    ax2.plot(x1, Lorxtofit(x1, fit_params), label='Fit 1',c="c")
    ax2.plot(x2, Lorytofit(x2, fit_params), label='Fit 2',c="r")        
    ax2.legend()
    plt.show(fig)
    
    parameters_dict = dict(fit_params)
    # Access the value of f_0
    f_0_value = parameters_dict['f_0'].value
    return(float(f_0_value))

def fittingforfreq(x,y,f):
    f_0 = 32.755e3
    A = 6.040484330735126e-5
    width = 4
    offset = 0
    angle = np.pi
    ############## param guess ###############
    div=100000 # I haven't added a way to record errors yet but this fitting method needs some values
    # Initialize the parameters
    params = Parameters()
    params.add('f_0', value=f_0)
    params.add('A', value=A)
    params.add('width', value=width, min=0)
    params.add('angle', value=angle,min=0,max=2*np.pi)
    params.add('offsetx', value=offset)
    params.add('offsety', value=offset)

    x1=np.array(list(filter(lambda x: not math.isnan(x), f)))
    y1=np.array(list(filter(lambda x: not math.isnan(x), x)))
    x2=x1
    y2=np.array(list(filter(lambda x: not math.isnan(x), y)))
    y1_err=np.array(list(filter(lambda x: not math.isnan(x), x/div)))
    y2_err=np.array(list(filter(lambda x: not math.isnan(x), x/div)))

    # Perform the simultaneous fit
    result = minimize(objective, params, args=(x1, y1, x2, y2, y1_err, y2_err))
    fit_params = result.params   
    parameters_dict = dict(fit_params)
    # Access the value of f_0
    f_0_value = parameters_dict['f_0'].value
    return(float(f_0_value))

In [ ]:
#A function to take a sweep of frequency and fit for the central frequency of resonance
def getfreq(start_frequency,end_frequency,num_points,delay,fig):
    # Run the sweep
    plt.rcParams["font.family"] = "Arial"
    do1d(frequency_parameter, start_frequency, end_frequency, num_points, delay, Amplitude_parameterx, Amplitude_parametery,
         write_period=0.1, show_progress=False, do_plot=False)
    dataset = load_by_run_spec(
    experiment_name=exp.name,
    sample_name=exp.sample_name,
    captured_run_id=exp.last_counter)
    x=dataset.get_parameter_data()["X_Amplitude"]["X_Amplitude"]
    y=dataset.get_parameter_data()["Y_Amplitude"]["Y_Amplitude"]
    f=dataset.get_parameter_data()["X_Amplitude"]["Frequency"]
    if fig==True:
        return(fittingforfreqfig(x,y,f))
    else:
        return(fittingforfreq(x,y,f))

In [ ]:
def getfreq2(start_frequency,end_frequency,num_points,delay):
    # Run the sweep
    plt.rcParams["font.family"] = "Arial"
    do1d(frequency_parameter, start_frequency, end_frequency, num_points, delay, Amplitude_parameterx, Amplitude_parametery, write_period=0.1, show_progress=False, do_plot=False)
    dataset = load_by_run_spec(
    experiment_name=exp.name,
    sample_name=exp.sample_name,
    captured_run_id=exp.last_counter)
    x=dataset.get_parameter_data()["X_Amplitude"]["X_Amplitude"]
    y=dataset.get_parameter_data()["Y_Amplitude"]["Y_Amplitude"]
    f=dataset.get_parameter_data()["X_Amplitude"]["Frequency"]
    r=np.sqrt(x**2+y**2)
    return(f[np.argmax(r, axis=0)])

In [ ]:
"""# Define the sweep range and other parameters
start_frequency = 32.70e3  # Hz
end_frequency = 32.835e3  # Hz
num_points = 60
delay=0.1
#A method I made for tracking the frequency by fitting a lorentzian to x and y measurements
print(getfreq(start_frequency,end_frequency,num_points,delay,True))"""

"""start_frequency = 32.70e3  # Hz
end_frequency = 32.835e3  # Hz
num_points = 60
delay=0.1
#A method I made for tracking the frequency by fitting a lorentzian to x and y measurements
#This version does not plot a graph
print(getfreq(start_frequency,end_frequency,num_points,delay,False))"""

"""start_frequency = 32.70e3  # Hz
end_frequency = 32.835e3  # Hz
num_points = 60
delay=0.1
#A method I made for tracking the frequency
#This method just finds the maximum value of r sqrt(x**2+y**2) and its frequency best to use this or both to track frequency
print(getfreq2(start_frequency,end_frequency,num_points,delay))"""